## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [4]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(1)
cv2.destroyAllWindows()
img_size = (img.shape[1],img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints,img_size,None,None)

## Apply a distortion correction to raw images.


In [5]:
def cal_undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

#### Debug distortion correction to raw images:


In [34]:
images = glob.glob('../camera_cal/calibration1.jpg')
for fname in images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, mtx, dist)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undistorted)
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


plt.savefig('undistort_output_mine.png')

#### close debug plots

In [18]:
plt.close('all')

## Use color transforms, gradients, etc., to create a thresholded binary image.

In [23]:
def pipeline(img, s_thresh=(120, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    binary = np.zeros_like(sxbinary)
    binary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 255
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return binary , color_binary

#### test color, gradient transforms:

In [47]:
test_images = glob.glob('../test_images/straight_lines2.jpg')
for fname in test_images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, mtx, dist)
    result , colored_result = pipeline(undistorted)
    plt.imshow(colored_result)
    plt.savefig('straight_lines2_thresh.png')
    im_undistorted_rgb = cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB)
    im_distorted_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(im_distorted_rgb)
    ax1.set_title('Original Image', fontsize=40)
    ax2.imshow(im_undistorted_rgb)
    ax2.set_title('Undistorted Result', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('straight_lines2.png')

#### close debug plots

In [45]:
plt.close('all')

## Apply a perspective transform to rectify binary image ("birds-eye view"):

#### extract perspective transform pixels from straight road by double clicking image

In [ ]:
def draw_circle(event,x,y,flags,param):
    global mouseX,mouseY
    if event == cv2.EVENT_LBUTTONDBLCLK:
        cv2.circle(img,(x,y),4,(255,0,0),-1)
        mouseX,mouseY = x,y
img = cv2.imread(test_images[len(test_images)-1])
undistorted = cal_undistort(img, mtx, dist)
cv2.namedWindow('img')
cv2.setMouseCallback('img',draw_circle)
while(1):
    cv2.imshow('img',img)
    k = cv2.waitKey(20) & 0xFF
    if k == 27:
        cv2.destroyAllWindows()
        break
    elif k == ord('a'):
        print(mouseX,mouseY)

#### Use hardcoded points from above to extract perspective transform

In [50]:
#these are my originals:
p1=[550,464]
p2=[743,461]
p3=[1279,685]
p4=[115,685]
#these were better:
p1=[585,460]
p2=[203,720]
p3=[1127,720]
p4=[695,460]
| Source        | Destination   |
|:-------------:|:-------------:|
| 585, 460      | 320, 0        |
| 203, 720      | 320, 720      |
| 1127, 720     | 960, 720      |
| 695, 460      | 960, 0        |

def get_perspective_matrix(p1,p2,p3,p4,undist):
    src = np.float32([p1,p2,p3,p4])
    #orig
    dst = np.float32([[0,0],[undist.shape[1],0],[undist.shape[1],undist.shape[0]],[0,undist.shape[0]]])
    #better ones from example writeup:
    dst = np.float32([[320,0],[320,720],[960,720],[960,0]])
    print(dst)
    M = cv2.getPerspectiveTransform(src, dst)
    return M

SyntaxError: invalid syntax (<ipython-input-50-8f7628b34782>, line 11)

#### corners unwarp - changes perspective of image

In [51]:
def corners_unwarp(undist , M):
    width,height,d = undist.shape
    warped = cv2.warpPerspective(undist, M, (height,width), flags=cv2.INTER_LINEAR)
    return warped

test warped perspective on straight line image:

In [57]:
img = cv2.imread(test_images[len(test_images)-1])
im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
undist = cal_undistort(im_rgb, mtx, dist)
cv2.circle(undist,(p1[0],p1[1]),4,(255,0,0),-1)
cv2.circle(undist,(p2[0],p2[1]),4,(255,0,0),-1)
cv2.circle(undist,(p3[0],p3[1]),4,(255,0,0),-1)
cv2.circle(undist,(p4[0],p4[1]),4,(255,0,0),-1)
perspective_M = get_perspective_matrix(p1,p2,p3,p4,undist)
top_down  = corners_unwarp(undist,perspective_M)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(undist)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(top_down)
ax2.set_title('Undistorted and Warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('warped_straight_lines_mine.png')

[[   0.    0.]
 [1280.    0.]
 [1280.  720.]
 [   0.  720.]]


close debug plots:

In [55]:
plt.close('all')

## Detect lane pixels and fit to find the lane boundary.

Test undistortion , perspectve transformation and thresholdin for lane pixels:

In [54]:
test_images = glob.glob('../test_images/*.jpg')
img = cv2.imread(test_images[len(test_images)-1])
im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
undist = cal_undistort(im_rgb, mtx, dist)
perspective_M = get_perspective_matrix(p1,p2,p3,p4,undist)
for fname in test_images:
    img = cv2.imread(fname)
    undist = cal_undistort(img, mtx, dist)
    top_down  = corners_unwarp(undist,perspective_M)
    binary_warped , colored_result = pipeline(top_down)
    top_down_rgb = cv2.cvtColor(top_down, cv2.COLOR_BGR2RGB)
    f, (ax1, ax2 ) = plt.subplots(2, 2, figsize=(24, 9))
    f.tight_layout()
    ax1[0].imshow(top_down_rgb)
    ax1[0].set_title('Original Image', fontsize=40)
    ax2[0].imshow(result)
    ax2[0].set_title('Pipeline Result', fontsize=40)
    ax1[1].imshow(colored_result)
    ax1[1].set_title('colored thresh Result', fontsize=40)
    ax2[1].imshow(img)
    ax2[1].set_title('Original', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

[[   0.    0.]
 [1280.    0.]
 [1280.  720.]
 [   0.  720.]]


close debug plots:

In [ ]:
plt.close('all')

#### Initial lane detection function (sliding windows histogram approach from course):

In [58]:
def find_lane_pixels_init(binary_warped):
    # Take a histogram of the bottom half of the image
    side_roi = 100
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,side_roi:binary_warped.shape[1]-side_roi], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)

    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 100

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = side_roi+leftx_base
    rightx_current = side_roi+rightx_base
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin# Update this
        win_xright_high = rightx_current + margin  # Update this
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        win_nonzero = binary_warped[win_y_low:win_y_high,leftx_current-margin:leftx_current+margin].nonzero()
        if ( len(good_left_inds) > minpix  ):
            leftx_current = leftx_current-margin+np.int(np.mean(win_nonzero[1]))
        win_nonzero=binary_warped[win_y_low:win_y_high,rightx_current-margin:rightx_current+margin].nonzero()
        if ( len(good_right_inds) > minpix  ):
            rightx_current = rightx_current-margin+np.int(np.mean(win_nonzero[1]))

        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2)
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2)
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

Test by fitting a polynomial to the lane pixels after sliding window histrogram detection:

In [65]:
def fit_init_polynomial_with_viz(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels_init(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit_params = np.polyfit(lefty,leftx,2)
    right_fit_params = np.polyfit(righty,rightx,2)
    print("leftfit params",left_fit_params)
    print("rightfit params",right_fit_params)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit_params[0]*ploty**2 + left_fit_params[1]*ploty + left_fit_params[2]
        right_fitx = right_fit_params[0]*ploty**2 + right_fit_params[1]*ploty + right_fit_params[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img


out_img = fit_init_polynomial_with_viz(binary_warped)

plt.imshow(out_img)
plt.savefig('first_fit.jpg')

leftfit params [-6.93871591e-05  4.86094218e-02  1.69402555e+02]
rightfit params [1.17880484e-06 3.07352674e-02 1.01906051e+03]


#### Fit polynmials to pixels from the detected lane lines

In [60]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    return left_fitx, right_fitx, left_fit , right_fit , ploty

#### Continous lane detection algo, takes pixels that are +- margin from previous iteration detected lane polynomial, in case not enought pixels are found initializes lane detection


In [61]:
leftx, lefty, rightx, righty, out_img = find_lane_pixels_init(binary_warped)
left_fitx, right_fitx, left_fit , right_fit, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
def search_around_poly(binary_warped , left_fit_params , right_fit_params ):
    margin = 100
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###

    left_lane_inds = ((nonzerox > (left_fit_params[0]*(nonzeroy**2) + left_fit_params[1]*nonzeroy + left_fit_params[2] - margin)) &
                      (nonzerox < (left_fit_params[0]*(nonzeroy**2) + left_fit_params[1]*nonzeroy + left_fit_params[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit_params[0]*(nonzeroy**2) + right_fit_params[1]*nonzeroy + right_fit_params[2] - margin)) &
                       (nonzerox < (right_fit_params[0]*(nonzeroy**2) + right_fit_params[1]*nonzeroy + right_fit_params[2] + margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit new polynomials
    left_fitx, right_fitx, left_fit_params , right_fit_params , ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin,
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin,
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    thresh = 100
    success = False
    print("lane inds: " , sum(right_lane_inds) , " left inds ",sum(left_lane_inds))
    if sum(right_lane_inds) > thresh and sum(left_lane_inds) > thresh:
        print("success")
        success = True
    ## End visualization steps ##
    return result , success , left_fitx, right_fitx , ploty , leftx , lefty , rightx , righty , left_fit_params , right_fit_params
# Run image through the pipeline
# Note that in your project, you'll also want to feed in the previous fits
result , success , left_fitx, right_fitx , ploty , leftx , lefty , rightx , righty , left_fit_params , right_fit_params = search_around_poly(binary_warped , left_fit , right_fit)
# View your output
plt.imshow(result)



lane inds:  9474  left inds  30144
success


#### Approximate real curvature from detected lane pixels

In [62]:
def measure_curvature_real(lefty,leftx,righty,rightx,img_shape):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    left_fit = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

    return left_curverad, right_curverad

#### Full process of lane detections, saves previous iteration lane polynomials and takes care of visualization

In [63]:

def process_image(image):
    undistorted_image = cal_undistort(image, mtx, dist)
    top_down  = corners_unwarp(undistorted_image,perspective_M)
    binary_warped , colored_result = pipeline(top_down)
    #print("search track: left fit", process_image.left_fit_params , " right fit " , process_image.right_fit_params)
    result , success , left_fitx, right_fitx , ploty , leftx , lefty , rightx , righty , process_image.left_fit_params , process_image.right_fit_params = search_around_poly(binary_warped , process_image.left_fit_params , process_image.right_fit_params )
    if not success:
        leftx, lefty, rightx, righty, out_img = find_lane_pixels_init(binary_warped)
        left_fitx, right_fitx, process_image.left_fit_params , process_image.right_fit_params, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
        print("after track: left fit", process_image.left_fit_params , " right fit " , process_image.right_fit_params)
    left_curverad, right_curverad = measure_curvature_real(lefty,leftx,righty,rightx,top_down.shape)
    font = cv2.FONT_HERSHEY_SIMPLEX
    curvature_info_string = "left curve is {} and right is {} m".format(left_curverad, right_curverad)
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    center_pic = top_down.shape[1]
    meters_from_mid = (((rightx[len(rightx)-1]+leftx[len(leftx)-1])/2)-(center_pic/2))*xm_per_pix
    dist_from_center_string = "dist from center is {} m".format(meters_from_mid)
    cv2.putText(undistorted_image, curvature_info_string, (10,30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(undistorted_image, dist_from_center_string, (10,60), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
    window_img = np.zeros_like(image)
    binary_mask = np.zeros_like(binary_warped)
    lane_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    lane_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx,ploty])))])
    lane_pts = np.hstack((lane_left, lane_right))
    cv2.fillPoly(window_img, np.int_([lane_pts]), (0,255, 0))
    binary_mask[lefty, leftx] = 255
    binary_mask[righty, rightx] = 100
    orig_perspective_binary_img = corners_unwarp(window_img,inv_perspective_M)
    orig_perspective_binary_mask = cv2.warpPerspective(binary_mask, inv_perspective_M, (binary_mask.shape[1],binary_mask.shape[0]), flags=cv2.INTER_LINEAR)
    #cv2.imshow("result top down",result)
    #cv2.waitKey(1)
    result = cv2.addWeighted(undistorted_image, 1.0, orig_perspective_binary_img, 0.3, 0)
    #print("shape " , orig_perspective_binary_mask.shape , " orig mask " , binary_mask.shape)
    result[orig_perspective_binary_mask==255]=[255,0,0]
    result[orig_perspective_binary_mask==100]=[0,0,255]
    cv2.imshow("movie",result)
    cv2.waitKey(1)
    return result
process_image.right_fit_params = [0,0,0]
process_image.left_fit_params = [0,0,0]

In [68]:
result = process_image(img)
result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
plt.imshow(result_rgb)
plt.savefig('full_vis.png')

lane inds:  9474  left inds  30140
success


In [64]:
# Import everything needed to edit/save/watch video clips
inv_perspective_M = np.linalg.pinv(perspective_M)
from moviepy.editor import VideoFileClip
challenge_output = '../output_images/challenge.mp4'
clip3 = VideoFileClip('../project_video.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

lane inds:  55  left inds  55
after track: left fit [-2.39250032e-04  3.33134431e-01  1.31292994e+02]  right fit  [-2.35534142e-04  3.76605643e-01  9.77863363e+02]
Moviepy - Building video ../output_images/challenge.mp4.
Moviepy - Writing video ../output_images/challenge.mp4

lane inds:  11066  left inds  35092
success
lane inds:  10640  left inds  33715
success
lane inds:  10067  left inds  33412
success
lane inds:  9399  left inds  32560
success
lane inds:  9239  left inds  32719
success
lane inds:  10036  left inds  32358
success
lane inds:  9797  left inds  32057
success
lane inds:  12624  left inds  33489
success
lane inds:  12496  left inds  33673
success
lane inds:  11571  left inds  33946
success
lane inds:  11100  left inds  33367
success
lane inds:  11126  left inds  33806
success
lane inds:  11200  left inds  35362
success
lane inds:  10931  left inds  34388
success
lane inds:  9996  left inds  33515
success
lane inds:  9287  left inds  32844
success
lane inds:  9307  left i